# Fine Tuning Workshop

The goal of this workshop is to simply a prompt to get chatGPT to write code into a given format.

## Install Dependencies

The first step is to install dependencies

In [2]:
pip install --upgrade openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.27.9
    Uninstalling openai-0.27.9:
      Successfully uninstalled openai-0.27.9
Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import openai
import os
import glob


## Orignal Prompt

The original prompt that we want to replace

>>>
 
According to the new user's task and our software designs listed below: 
Task: "Create a pong game".
We have decided to complete the task through a executable software with
    a static html website. As the software develoepr, 
    to satisfy the new user's demands, you should write one or multiple 
    files and make sure that every detail of the architecture is, in the end, 
    implemented as code.
Think step by step and reason yourself to the right decisions to make sure we get it right.
You will output the content of the complete code. Each file must strictly follow a 
markdown code block format, where the following tokens must be replaced such that 
"FILENAME" is the lowercase file name including the file extension, "LANGUAGE" 
is the programming language and "CODE" is the original code.  The code should have
no comments, and you must respons with only the code and file-name... nothing

[FILENAME]
```LANGUAGE
CODE
```

For example:

[index.html]
```html
<!DOCTYPE html>
<html>
  <head>
  </head>
</html>
```

You will start with the "[index.html]" file, then go to the ones that are imported by that file, and so on.
Please note that the code should be fully functional. Ensure to implement all functions.

```

## Problems with Prompt

The prompt is very long, and takes a lot of examples.  It would be better to "show" as opposed to tell.

## Example of first prompt

We will run the first prompt to see the output

In [9]:
prompt = """
According to the new user's task and our software designs listed below: 
Task: "Create a pong game".
We have decided to complete the task through a executable software with
    a static html website. As the software develoepr, 
    to satisfy the new user's demands, you should write one or multiple 
    files and make sure that every detail of the architecture is, in the end, 
    implemented as code.
Think step by step and reason yourself to the right decisions to make sure we get it right.
You will output the content of the complete code. Each file must strictly follow a 
markdown code block format, where the following tokens must be replaced such that 
"FILENAME" is the lowercase file name including the file extension, "LANGUAGE" 
is the programming language and "CODE" is the original code.  The code should have
no comments, and you must respons with only the code and file-name... nothing

[FILENAME]
```LANGUAGE
CODE
```

For example:

[index.html]
```html
<!DOCTYPE html>
<html>
  <head>
  </head>
</html>
```

You will start with the "[index.html]" file, then go to the ones that are imported by that file, and so on.
Please note that the code should be fully functional. Ensure to implement all functions.

```

## Problems with Prompt

The prompt is very long, and takes a lot of examples.  It would be better to "show" as opposed to tell.
"""

In [10]:
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a web developer"},
    {"role": "user", "content": prompt}
  ]
)

print(completion.choices[0].message.content)

[index.html]
```html
<!DOCTYPE html>
<html>
  <head>
    <title>Pong Game</title>
    <style>
      #pong {
        position: absolute;
        width: 100%;
        height: 100%;
        background: black;
        overflow: hidden;
      }
    </style>
  </head>
  <body>
    <div id="pong"></div>
    <script src="pong.js"></script>
  </body>
</html>
```

[pong.js]
```javascript
const pong = document.getElementById('pong');
const context = pong.getContext('2d');

// Create the Paddle
const paddleWidth = 10;
const paddleHeight = 100;
const leftPaddle = {
  x: 0,
  y: pong.height / 2 - paddleHeight / 2,
  width: paddleWidth,
  height: paddleHeight,
  color: 'white',
  dy: 4
};
const rightPaddle = {
  x: pong.width - paddleWidth,
  y: pong.height / 2 - paddleHeight / 2,
  width: paddleWidth,
  height: paddleHeight,
  color: 'white',
  dy: 4
};

function drawPaddle(x, y, width, height, color) {
  context.fillStyle = color;
  context.fillRect(x, y, width, height);
}

function update() {
  co

## Problems with response

The given response gives more detail than we want... we only want the code, but it gives commentary about the code... We could probably get this eliminated with prompt engineering, but we can try fine-tuning

## Prepare Example data

Example data is prepared in the `example_data` folder.  The goal is to be able to tell chat GPT to write code with a small prompt, and give data back in the following format

[filename.ext]
```code
CODE HERE
```

In [4]:
files = glob.glob("example_data/*")
files

['example_data/Joke generator.txt',
 'example_data/A template restaurant website.txt',
 'example_data/A sudoku game.txt',
 'example_data/An HTML color picker.txt',
 'example_data/breakout game.txt',
 'example_data/Landing page template.txt',
 'example_data/2048 game.txt',
 'example_data/A bbc website clone.txt',
 'example_data/A hangman game.txt',
 'example_data/HTML5 calculator.txt',
 'example_data/A pong game.txt',
 'example_data/A flappy bird game.txt',
 'example_data/A tetris game.txt',
 'example_data/A snake game.txt',
 'example_data/A music shop page.txt',
 'example_data/A photo gallery template.txt']

## Save data as a JSONL file

Here we will take the data and write it as a JSONL file

In [5]:
system_content = "You are a developer for a company"

lines = []
for file in files:
    name = file.split("/")[-1].split(".")[0]
    user_content = name
    with open(file, 'r') as fi:
        assistant_content = json.dumps(fi.read())
    line = "{\"messages\": [{\"role\": \"system\", \"content\": \"" + system_content + "\"}, {\"role\": \"user\", \"content\": \"" + user_content + "\"}, {\"role\": \"assistant\", \"content\":"  + assistant_content + "}]}"
    lines.append(line)

In [6]:
lines

['{"messages": [{"role": "system", "content": "You are a developer for a company"}, {"role": "user", "content": "Joke generator"}, {"role": "assistant", "content":"[index.html]\\n```html\\n<!DOCTYPE html>\\n<html lang=\\"en\\">\\n<head>\\n    <meta charset=\\"UTF-8\\">\\n    <meta name=\\"viewport\\" content=\\"width=device-width, initial-scale=1.0\\">\\n    <title>Joke Generator</title>\\n    <link rel=\\"stylesheet\\" href=\\"styles.css\\">\\n</head>\\n<body>\\n    <div class=\\"container\\">\\n        <h1>Joke Generator</h1>\\n        <p id=\\"joke\\">Click the button to get a joke.</p>\\n        <button onclick=\\"generateJoke()\\">Generate Joke</button>\\n    </div>\\n    <script src=\\"script.js\\"></script>\\n</body>\\n</html>\\n```\\n\\n[styles.css]\\n```css\\nbody {\\n    font-family: Arial, sans-serif;\\n    display: flex;\\n    justify-content: center;\\n    align-items: center;\\n    height: 100vh;\\n    background-color: #f3f4f6;\\n}\\n\\n.container {\\n    text-align: cen

In [7]:
with open('developer-initialize.jsonl', 'w') as f:
    for line in lines:
        f.write(line)
        f.write("\n")


## Upload Model

While it's possible to use the API, it's easier to simply upload the model here: https://platform.openai.com/finetune

## Run the model

In [8]:
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="ft:gpt-3.5-turbo-0613:personal::89edVQIv",
  messages=[
    {"role": "system", "content": "You are a web developer"},
    {"role": "user", "content": "Create a pong game"}
  ]
)

print(completion.choices[0].message.content)

[index.html]
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Pong Game</title>
    <link rel="stylesheet" href="styles.css">
</head>
<body>
    <canvas id="pongCanvas" width="800" height="400"></canvas>
    <script src="script.js"></script>
</body>
</html>
```

[styles.css]
```css
body {
  display: flex;
  justify-content: center;
  align-items: center;
  height: 100vh;
  background-color: #000;
}

canvas {
  border: 1px solid white;
}
```

[script.js]
```javascript
const canvas = document.getElementById('pongCanvas');
const context = canvas.getContext('2d');

const paddleWidth = 10, paddleHeight = 100;
let paddle1Y = (canvas.height - paddleHeight) / 2;
let paddle2Y = (canvas.height - paddleHeight) / 2;
let ballX = canvas.width / 2, ballY = canvas.height / 2;
let ballSpeedX = 3, ballSpeedY = 3;
let playerScore = 0, computerScore = 0;

canvas.addEventListener('mousemove', fun